# Basic Flowsheet Build Tutorial

This tutorial will go through the basic workflow to create a WaterTAP model with a single unit process (pump).

0. Required imports
1. Create model
2. Add flowsheet
3. Add property model
4. Add pump model
5. Set operating conditions
6. Scaling model
7. Initialize model
8. Solve model

## Step 0: Import necessary packages

### Any WaterTAP model will begin with importing the relevant components from Pyomo, IDAES, and WaterTAP.

From Pyomo:
- Concrete model
- units
- Helper function(s) to check solver status
- Helper function to get value of variables

From IDAES:
- Flowsheet block
- Helper function to check degrees of freedom
- Helper function to scale model

From WaterTAP:
- Property models
- Unit models
- Solver


<div class="alert alert-block alert-info">
<b>Note:</b> Both <code>ConcreteModel</code> and <code>FlowsheetBlock</code> are required imports for any WaterTAP model.
</div>

In [ ]:
# Pyomo imports
from pyomo.environ import (
    ConcreteModel,
    check_optimal_termination,
    assert_optimal_termination,
    value,
    units as pyunits,
)

# IDAES imports
from idaes.core import FlowsheetBlock
from idaes.core.util.model_statistics import degrees_of_freedom
from idaes.core.util.scaling import calculate_scaling_factors

# WaterTAP imports
from watertap.core.solvers import get_solver
from watertap.property_models.seawater_prop_pack import SeawaterParameterBlock
from watertap.unit_models.pressure_changer import Pump

## Step 1: Create base model

Pyomo's `ConcreteModel` is the basis for any WaterTAP model.

No arguments are required. Convention is to call this `m`.

In [ ]:
m = ConcreteModel()

## Step 2: Add flowsheet block

The [`FlowsheetBlock`](https://idaes-pse.readthedocs.io/en/stable/reference_guides/core/flowsheet_block.html#) from IDAES forms the base for all WaterTAP system models.

All model components (unit models, property models, variables, constraints, etc.) are built upon the flowsheet block.

The flowsheet block will create the time domain for the flowsheet. We set `dynamic=False` when creating the `FlowsheetBlock` to represent a steady-state flowsheet.

(Note: `dynamic=False` is the default argument, but excluding it will raise a warning.)

In [ ]:
m.fs = FlowsheetBlock(dynamic=False)

# Step 3: Add property model

With the flowsheet created, we can add any sub-models required for the WaterTAP system model.

The property model used will be dictated by the models on the flowsheet. In this demonstration, we are using the WaterTAP [seawater property model](https://watertap.readthedocs.io/en/stable/technical_reference/property_models/seawater.html).

Creating the `SeawaterParameterBlock` does not require any arguments. However, other property models can have many required and optional arguments to specify the model.

In [ ]:
m.fs.properties = SeawaterParameterBlock()

# Step 4: Add unit model

After adding the property model, we can add the unit models to the flowsheet. In this demonstration, we are adding a single [pump model](https://watertap.readthedocs.io/en/stable/technical_reference/unit_models/pump.html).

Adding any unit model requires specifying the property package via the `property_package` argument. Here, we are specifying that the property model to be used with our pump model is `m.fs.properties` that we just added to the flowsheet.

In [ ]:
m.fs.pump = Pump(
    property_package=m.fs.properties,
)

# Step 5: Set operating conditions

Now that we have added all the necessary models to the flowsheet, we can specify the operating conditions.

First, we check the degrees of freedom that need to be specified prior to solving the model. Degrees of freedom for individual unit models can be found on the WaterTAP documentation.


In [ ]:
print("Degrees of freedom =", degrees_of_freedom(m))

### The degrees of freedom for the pump model include:
- the change in pressure from inlet to outlet
- pump efficiency
- inlet stream state variables

The inlet stream conditions to any unit model are defined by the property model state variables. For the pump model, the inlet and outlet properties are managed by a [control volume](https://idaes-pse.readthedocs.io/en/stable/reference_guides/core/control_volume_0d.html#module-idaes.core.base.control_volume0d).

We can use the `.display()` method on the pump unit model control volume. Using this method on any block will automatically go through all the sub-blocks and show all the variables, constraints, and objectives currently on each block.

In [ ]:
m.fs.pump.control_volume.display()

### This reveals that on `m.fs.pump.control_volume` are two sub-blocks called `properties_in` and `properties_out`. 

Each of these are indexed to `0`, which is the default time-index for steady-state models.

To define the inlet stream to our pump, we only need to specify the state variables for `properties_in`. The state variables are defined by the property model used for the unit model (here that is the `SeawaterParameterBlock`) but are nearly always:
- `pressure`: pressure of inlet stream (Pa)
- `temperature`: temperature of (K)
- `flow_mass_phase_comp["Liq", "H2O"]`: mass flow rate of water (kg/s)
- `flow_mass_phase_comp["Liq", "TDS"]`: mass flow rate of TDS (kg/s)

<div class="alert alert-block alert-info">
<b>Note:</b> The naming of property variables in WaterTAP property models follow the <a href="https://idaes-pse.readthedocs.io/en/latest/explanations/conventions.html#standard-naming-format">naming conventions used in IDAES</a>.
</div>

In [ ]:
m.fs.pump.control_volume.properties_in[0].pressure.fix(101325)  # Pa
m.fs.pump.control_volume.properties_in[0].temperature.fix(298.15)  # K
m.fs.pump.control_volume.properties_in[0].flow_mass_phase_comp["Liq", "H2O"].fix(
    1
)  # kg/s
m.fs.pump.control_volume.properties_in[0].flow_mass_phase_comp["Liq", "TDS"].fix(
    0.1
)  # kg/s

print("Degrees of freedom =", degrees_of_freedom(m))

### With the inlet stream specified, the remaining two degrees of freedom are variables that are built directly on the pump block itself (rather than on the control volume sub-block):

In [ ]:
m.fs.pump.deltaP.fix(500000)  # Pa
m.fs.pump.efficiency_pump.fix(0.8)

print("Degrees of freedom =", degrees_of_freedom(m))

# Step 6: Scaling the model

Prior to model initialization and solving, the variables and constraints on the model must be properly scaled.

Model scaling is a topic that will be covered in more detail in a future class, but here we present the helper function `calculate_scaling_factors` that will use default scaling factors to scale the variables on the model.

<div class="alert alert-block alert-info">
<b>Note:</b> Since we did not set default scaling factors for some of our variables, this will result in printed warnings.
</div>

In [ ]:
calculate_scaling_factors(m)

# Step 7: Initializing the model

Before solving the model, we run an initialization routine that will set model variables to initial values based on the operating conditions we have specified in the previous step.

Every model has a different initialization routine that was implemented by the model developer, but they can all be called with the `initialize()` method.

In [ ]:
m.fs.pump.initialize()

# Step 8: Solving the model

After initialization, we are ready to solve the model.

Solving is done by passing the model to a *solver*. WaterTAP uses a custom implementation of the IPOPT solver. An instance of the WaterTAP solver is returned from the `get_solver()` function that was imported above.

In [ ]:
solver = get_solver()
results = solver.solve(m)

### Checking the termination status of the solve

We want to know if the solve resulted in an optimal termination. This can be done in a few ways:
- using the `assert_optimal_termination` or `check_optimal_termination` function
- printing the solver status from the `results`

In [ ]:
# This will raise an error if the model did not solve optimally
assert_optimal_termination(results)

# Alternatively, we can check the termination status without raising an error
termination_status = check_optimal_termination(results)
print("Optimal termination:", termination_status)

# Printing termination condition from results
print("Solver termination condition:", results.solver.termination_condition)

# Accessing results

The values of all variables on the model can be accessed directly by using the `value()` function.

In [ ]:
pump_work = value(m.fs.pump.work_mechanical[0])

print(f"Pump work: {pump_work:.2f} W")

Alternatively, most unit models in WaterTAP and IDAES have a `report()` method.

In [ ]:
m.fs.pump.report()

# Build function

It is convenient to put some of the steps for creating our model into a single `build` function that returns the model.

In [ ]:
def build():
    m = ConcreteModel()

    m.fs = FlowsheetBlock(dynamic=False)

    m.fs.properties = SeawaterParameterBlock()

    m.fs.pump = Pump(
        property_package=m.fs.properties,
    )

    calculate_scaling_factors(m)

    return m

# Working with quantities and units

WaterTAP makes use of a package called [Pint](https://pint.readthedocs.io/en/stable/) to handle dimensional quantities, which we have imported as `pyunits`.

Variables, parameters, and expressions in WaterTAP models have physical units associated with them to ensure dimensional consistency.

But `pyunits` can be used just as easily to define a quantity outside of WaterTAP. A quantity is the product of the magnitude and the associated dimensional units.

In [ ]:
conc = 35 * pyunits.g / pyunits.L
flow_vol = 2e6 * pyunits.gallons / pyunits.day

print(flow_vol)
# You can also use common prefixes to the units to indicate order of magnitude
# e.g., u = "micro" = 1E-6
flow_vol = 2 * pyunits.ugallons / pyunits.day
print(flow_vol)
# M = "mega" = 1E6
flow_vol = 2 * pyunits.Mgallons / pyunits.day
print(flow_vol)

## Converting units

You can convert from one set of units to another using the `convert` method and access the units of a quantity with the `get_units` method

In [ ]:
flow_vol = 1 * pyunits.Mgallons / pyunits.day
flow_vol2 = pyunits.convert(flow_vol, to_units=pyunits.m**3 / pyunits.s)
print(f"Flow volume in MGD = {value(flow_vol):.2f} {pyunits.get_units(flow_vol)}")
print(f"Flow volume in m3/s = {value(flow_vol2):.6f} {pyunits.get_units(flow_vol2)}")

## Creating new quantities

Units of quantities will carry through to define a new quantity with new units

In [ ]:
rho = 1000 * pyunits.g / pyunits.liter  # density of water

flow_mass_water1 = flow_vol * rho
flow_mass_water2 = pyunits.convert(flow_mass_water1, to_units=pyunits.kg / pyunits.s)

flow_mass_tds1 = flow_vol * conc
flow_mass_tds2 = pyunits.convert(flow_mass_tds1, to_units=pyunits.kg / pyunits.s)

print(
    f"Mass flow water 1 = {value(flow_mass_water1):.2f} {pyunits.get_units(flow_mass_water1)}"
)
print(
    f"Mass flow water 2 = {value(flow_mass_water2):.2f} {pyunits.get_units(flow_mass_water2)}"
)
print(
    f"Mass flow TDS 1 = {value(flow_mass_tds1):.2f} {pyunits.get_units(flow_mass_tds1)}"
)
print(
    f"Mass flow TDS 2 = {value(flow_mass_tds2):.2f} {pyunits.get_units(flow_mass_tds2)}"
)

### Use `pyunits` with WaterTAP state variables

The default units for WaterTAP state variables are:
- kg/s for mass flow rates
- Pascal for pressure
- Kelvin for temperature

When defining state variables, using magnitudes without units attached will assume the units to be the default units. 

However, if you attach units, WaterTAP will automatically convert to the default units. 

In the example below, we define our pressure in psi, change in pressure in mmHg, temperature in Rankine, mass flow of water in lb/hr, and mass flow of TDS in grains/year. After fixing our state variables with these quantities, they are automatically converted to the proper units.

<div class="alert alert-block alert-info">
<b>Note:</b> For temperature, you can only use absolute units (Kelvin and Rankine) and not relative units (Celsius and Fahrenheit).
</div>


In [ ]:
m = build()

pressure = 800 * pyunits.psi
temperature = 535 * pyunits.degR
flow_mass_water = 5000 * pyunits.lb / pyunits.hr
flow_mass_tds = 1.239e10 * pyunits.gr / pyunits.year

deltaP = 50234 * pyunits.mmHg

m.fs.pump.control_volume.properties_in[0].pressure.fix(pressure)  # Pa
m.fs.pump.control_volume.properties_in[0].temperature.fix(temperature)  # K
m.fs.pump.control_volume.properties_in[0].flow_mass_phase_comp["Liq", "H2O"].fix(
    flow_mass_water
)  # kg/s
m.fs.pump.control_volume.properties_in[0].flow_mass_phase_comp["Liq", "TDS"].fix(
    flow_mass_tds
)  # kg/s

print(
    f"\nModel pressure = {value(m.fs.pump.control_volume.properties_in[0].pressure):.2f} {pyunits.get_units(m.fs.pump.control_volume.properties_in[0].pressure)}"
)
print(
    f"Model temperature = {value(m.fs.pump.control_volume.properties_in[0].temperature):.2f} {pyunits.get_units(m.fs.pump.control_volume.properties_in[0].temperature)}"
)
print(
    f"Model mass flow of water = {value(m.fs.pump.control_volume.properties_in[0].flow_mass_phase_comp['Liq', 'H2O']):.2f} {pyunits.get_units(m.fs.pump.control_volume.properties_in[0].flow_mass_phase_comp['Liq', 'H2O'])}"
)
print(
    f"Model mass flow of TDS = {value(m.fs.pump.control_volume.properties_in[0].flow_mass_phase_comp['Liq', 'TDS']):.2f} {pyunits.get_units(m.fs.pump.control_volume.properties_in[0].flow_mass_phase_comp['Liq', 'TDS'])}"
)